In [1]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import platform
import random
import datetime
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import timm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import average_precision_score, cohen_kappa_score, matthews_corrcoef, precision_recall_fscore_support


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [3]:
dt = datetime.datetime.now().strftime("%m.%d_%H-%M-%S")
wandb_project_name = "emo_rec" # "emo_rec" dirty
experiment_name = f'{dt}' # model_file_name
run_desc=f""

torch.backends.cudnn.deterministic = True


folder_with_data = 'train-5000-190-14000resol'
# folder_with_data = '35887-48x48-fer-2013'
# folder_with_data = 'train-5000-190-35000-14000resol'
# folder_with_data = 'train-5000-190'
# folder_with_data = '35887-48x48-emotion-detection'
# folder_with_data = 'debug'

model_name = 'vgg19'
# model_name = 'resnet50'
# model_name = 'vgg_face_dag'
# model_name = 'seresnet50__ra2_in1k' # жёстко переобучился
# model_name = 'seresnet50__a1_in1k' # круто медленно учится
# model_name = 'seresnextaa101d_32x8d__sw_in12k_ft_in1k' 
# model_name = 'resnext101_32x32d__fb_wsl_ig1b_ft_in1k' # 15.7/16 Gb GPU mem при bs=15
# model_name = 'vit_large_patch14_clip_224.openai_ft_in12k_in1k'
# model_name = 'resnext101_32x4d.fb_swsl_ig1b_ft_in1k'
# model_name = 'resnext101_32x4d.fb_ssl_yfcc100m_ft_in1k'
# model_name = 'seresnext101_32x4d.gluon_in1k'


batch_size = 1 # 100 25 15 12 6 1 ########### 15 15 15 15 15 (next на 14 или 12 вроде качество лучше) 
lr = 0.000225 # 00025 00015 000175 000225
 #  0.001

optim_name = 'SGD' # SGD  Adam  NAdam    Adamax > RAdam
# betas=(0.9, 0.999)
# eps=1e-08
momentum = 0.9


start_epoch = 0 # !!!
num_epochs = 10 # 40

min_acc_to_save = 0.745
max_loss_to_save = 1.75


num_classes = 8 # !!!

path_to_pr_model = r''
# path_to_pr_model = r'/kaggle/input/k01-31-12-56-04/k01.31 12-56-04.pth'
# path_to_pr_model = r'/kaggle/input/vgg-face-dag/vgg_face_dag.pth'



data_transforms = {
    folder_with_data: transforms.Compose([
        transforms.RandomHorizontalFlip(), # Случайное горизонтальное отражение изображения (зеркальное отражение).
        transforms.RandomRotation(degrees=10), # Случайный поворот изображения на заданный угол в градусах.
        transforms.ColorJitter(brightness=0.30, contrast=0.30), # Случайное изменение яркости и контраста изображения.
        transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
        transforms.RandomApply([transforms.GaussianBlur(3)], p=0.25), # Добавление размытия

        transforms.Resize((224, 224)), # Измените размер изображения на 224x224
        transforms.ToTensor(), # Преобразуйте изображение в тензор
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], # Нормализация по средним значениям и стандартным отклонениям ImageNet
            std=[0.229, 0.224, 0.225]
        )
    ]),
}




target_names = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sad", "surprise"] # , "uncertain", ] # !!

train_test_split = 0.25

wandb_local_dir = r"../"                                         

In [4]:
import torch
import torch.nn as nn

class vgg_face_dag(nn.Module):

    def __init__(self):
        super(vgg_face_dag, self).__init__()
        
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)
        self.softmax8 = nn.Softmax()

        
    def forward(self, x0):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        
        x38 = self.fc8(x37)
        x39 = self.softmax8(x38)
        return x39

    
def vgg_face_dag_init(weights_path=None, **kwargs):
    """
    load imported model instance

    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = vgg_face_dag()
    
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model


# vgg_face_dag_init('/kaggle/input/vgg-face-dag/vgg_face_dag.pth')

In [5]:

if platform.system() == 'Windows':
    data_dir = '../data'
    models_dir = '../models'
else:
    data_dir = '/kaggle/input'
    experiment_name = 'k' + experiment_name
    wandb_local_dir = '/kaggle/working/'
    models_dir = '/kaggle/working/'




wandb.init(
    project=wandb_project_name, 
    name=experiment_name, 
    config={
        "dataset": folder_with_data,
        'data_transforms': data_transforms,
        "architecture": model_name,
        
        "train_test_split": 0.25,
        
        "epochs": num_epochs,
        "batch_size": batch_size,
        
        "learning_rate": lr,
        "momentum": momentum,
        
        'optim': optim_name,
    },
    dir = wandb_local_dir,
    notes=run_desc, # комментарии к запуску
    # group="Группа_экспериментов", # несколько запусков под общим именем группы
    # tags=wandb_tags, # wandb_tags = ["test", "try"]
    # resume="идентификатор_вашего_запуска" # возобновляет ранее завершенный запуск
    # entity="lev", 
    )


wandb: Currently logged in as: golovninlev. Use `wandb login --relogin` to force relogin


In [6]:
val_titile = 'val'
train_title = 'train'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
# if not torch.cuda.is_available(): 
#     import torch_xla
#     import torch_xla.core.xla_model as xm
#     # for TPU
#     device = xm.xla_device()
#     torch.set_default_tensor_type('torch.FloatTensor')
print(device)




# !3 train and test split
# Создаем загрузчики данных для обучения и валидации
image_datasets = {x: datasets.ImageFolder(root=data_dir + '/' + x, transform=data_transforms[x]) for x in [folder_with_data]}
# dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in [folder_with_data]}

# Предположим, у вас есть dataset, который вы загружаете с помощью DataLoader
dataset = image_datasets[folder_with_data]

# Определите размер тестового набора (например, 20% данных)
test_size = int(train_test_split * len(dataset))

# Определите размер обучающего набора
train_size = len(dataset) - test_size

# Разделите данные на обучающий и тестовый наборы, учитывая пропорции классов
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


dataloaders=dict({folder_with_data: DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4), 
                  val_titile: DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)})

# print(len(image_datasets[folder_with_data]), len(dataloaders[folder_with_data]), len(dataloaders[val_titile])) # 5710 429 143
# !3                                                     





# !3 взвешивание дизбаланса классов + Loss
if folder_with_data == 'train-5000-190-35000-14000resol':
    weights = torch.tensor([0.0228, 0.7191, 0.1614, 0.0286, 0.0115, 0.0150, 0.0148, 0.0267]) 
elif folder_with_data == 'train-5000-190-14000resol':
    weights = torch.tensor([0.0457, 0.4470, 0.2216, 0.1723, 0.0196, 0.0222, 0.0216, 0.0500])
elif folder_with_data == 'train-5000-190':
    weights = torch.tensor([0.0756, 0.3257, 0.1438, 0.1132, 0.0470, 0.1228, 0.0895, 0.0824])
elif folder_with_data == 'debug':
    weights = torch.tensor([0.1445, 0.3210, 0.2064, 0.2064, 0.0672, 0.0153, 0.0159, 0.0235])
elif folder_with_data == '35887-48x48-emotion-detection':
    weights = torch.tensor([0.0685, 0.9000, 0.6320, 0.0660, 0.0377, 0.0548, 0.0561, 0.0849]) # contempt = 0
elif folder_with_data == '35887-48x48-fer-2013':
    weights = torch.tensor([0.0692, 0.9000, 0.6260, 0.0679, 0.0387, 0.0556, 0.0568, 0.0858])
else:
    # Подсчет количества примеров каждого класса в обучающем наборе данных
    class_counts = torch.bincount(torch.cat([torch.tensor(target).view(-1) for _, target in dataloaders[folder_with_data].dataset], dim=0)) 
    class_weights = 1 / class_counts.float()
    weights = class_weights / class_weights.sum()
    
    print(f'''elif folder_with_data == '{folder_with_data}':
    weights = torch.{weights}''')

weights = weights.to(device)


criterion = nn.CrossEntropyLoss(weight=weights) 
# !3                                                     





# !4 model + optimizer
# Редактируем последний слой для задачи классификации на 8 классов

if model_name == 'resnext101_32x32d__fb_wsl_ig1b_ft_in1k':
    model = timm.create_model('resnext101_32x32d.fb_wsl_ig1b_ft_in1k', pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

if model_name == 'seresnextaa101d_32x8d__sw_in12k_ft_in1k':
    model = timm.create_model('seresnextaa101d_32x8d.sw_in12k_ft_in1k', pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

if model_name == 'seresnet50__a1_in1k':
    model = timm.create_model('seresnet50.a1_in1k', pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
if model_name == 'seresnet50__ra2_in1k':
    model = timm.create_model('seresnet50.ra2_in1k', pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
if model_name == 'resnext101_32x4d.fb_swsl_ig1b_ft_in1k':
    model = timm.create_model('resnext101_32x4d.fb_swsl_ig1b_ft_in1k', pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    
if model_name == 'seresnextaa101d_32x8d__sw_in12k_ft_in1k' \
        or model_name == 'seresnet50__a1_in1k' \
        or model_name == 'seresnet50__ra2_in1k':
    data_config = timm.data.resolve_model_data_config(model)
    transforms = timm.data.create_transform(**data_config, is_training=True)
    print(transforms)
    data_transforms[folder_with_data] = transforms
    

if model_name == 'resnet50':
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

if model_name == 'vgg19':
    model = models.vgg19(pretrained=True)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

if model_name == 'vgg_face_dag':
    model = vgg_face_dag_init(path_to_pr_model)
    model.fc8 = nn.Linear(model.fc8.in_features, num_classes)


if path_to_pr_model != '':
    if path_to_pr_model[-4:] == '.pth' and model_name != 'vgg_face_dag':
        model.load_state_dict(torch.load(path_to_pr_model)) # обученная модель для классификации эмоций

    if path_to_pr_model[-4:] == '.pkl':
        model = joblib.load(path_to_pr_model)
        if model_name == 'resnet50':
            model.fc = nn.Linear(model.fc.in_features, num_classes)
        if model_name == 'vgg19':
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)



model.to(device)
if optim_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum) # SGD
    
if optim_name == 'Adam':
#     optimizer = optim.Adam(model.parameters(), lr=lr, betas=betas, eps=eps)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
if optim_name == 'NAdam':
    optimizer = optim.NAdam(model.parameters(), lr=lr)
    
if optim_name == 'RAdam':
    optimizer = torch.optim.RAdam(model.parameters(), lr=lr)
    
if optim_name == 'Adamax':
    optimizer = torch.optim.Adamax(model.parameters(), lr=lr)

print(device)

cuda:0


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:01<00:00, 327MB/s]  


cuda:0


In [ ]:
for epoch in range(start_epoch, num_epochs):
    for phase in [folder_with_data, val_titile]:
        
        if phase == folder_with_data:
            model.train()
            phase_name = train_title
        else:
            model.eval()
            phase_name = val_titile

        running_loss = 0.0
        corrects = 0
        all_labels = []
        all_preds = []


        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == folder_with_data):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == folder_with_data:
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
            
            # Сохраняем истинные метки и предсказания для вычисления F1-Score
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())



        epoch_loss = running_loss / (len(dataloaders[phase]) * batch_size)
        # epoch_acc = (corrects.data.cpu() * 1.0) / (len(dataloaders[phase]) * batch_size)
        
        epoch_acc = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        f1 = f1_score(all_labels, all_preds, average='weighted')
        # roc_auc = roc_auc_score(all_labels, all_preds, average='weighted', multi_class='ovr')
        
        # average_precision = average_precision_score(all_labels, all_preds, average='weighted')
        # cohen_kappa = cohen_kappa_score(all_labels, all_preds)
        # mc = matthews_corrcoef(all_labels, all_preds, average='weighted')
        # prfs = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        
        if phase == val_titile and epoch_acc > min_acc_to_save and epoch_loss < max_loss_to_save:
            torch.save(model.state_dict(), f'{models_dir}{experiment_name}_e{epoch}.pth')
#             import pickle
#             with open(f'{models_dir}{experiment_name}_e{epoch}.pkl', 'wb') as f:
#                 pickle.dump(model, f)

        if epoch != num_epochs - 1:
            wandb.log({phase_name: {
                'loss': epoch_loss, 'acc': epoch_acc, 'precision': precision, 'recall': recall, 'f1': f1, 
                # 'classification_report': wandb_table, 
                }}, step=epoch)
        else:
#             class_rep_dict = classification_report(all_labels, all_preds, target_names=target_names,
#                 output_dict=True)
#             class_rep_dict.pop('accuracy')
#             class_rep_df = pd.DataFrame().from_dict(class_rep_dict).T.reset_index().round(2)
#             wandb_table = wandb.Table(data=class_rep_df, 
#                         columns=["class", "precision", "recall", "f1-score", "support"])
            
            conf_matrix = confusion_matrix(all_labels, all_preds)
            
            plt.figure()
            cm = confusion_matrix(y_true=all_labels, y_pred=all_preds)
            plot_cm = sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
            plt.xlabel("Predicted")
            plt.ylabel("True")
            plt.title(f"Confusion Matrix - {phase_name}")
            
            
            wandb.log({phase_name: {
                # 'epoch': epoch, 
                'loss': epoch_loss, 'acc': epoch_acc, 'precision': precision, 'recall': recall, 'f1': f1, 
#                 'classification_report': wandb_table, 
                'confusion_matrix': wandb.Image(plot_cm) # wandb.Image(img[0].numpy()*255)
                # 'average_precision': average_precision, 'cohen_kappa': cohen_kappa, 'matthews_corrcoef': mc,
                # 'precision_recall_fscore_support': prfs
                }}, step=epoch)

In [8]:
wandb.finish()

In [9]:
torch.save(model.state_dict(), f'{models_dir}{experiment_name}_e{epoch}.pth')

In [10]:
1

1